In [114]:
#import libraries

import sqlite3

import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn import preprocessing, linear_model

In [169]:
# data inlezen sql

dbName = "rest_server/db.sqlite3"
tableName = "bmi"

dbConnection = sqlite3.connect(dbName)
# How to convert sqlite3 object to pd dataframe?
dfFromDB = pd.read_sql_query(f"SELECT * FROM {tableName}", dbConnection)
pd.set_option('display.max_columns', 10)

df = dfFromDB.drop(['id'], axis=1)

df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4084 entries, 0 to 4083
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   genetic   4084 non-null   float64
 1   exercise  4084 non-null   float64
 2   smoking   4084 non-null   float64
 3   alcohol   4084 non-null   float64
 4   lifespan  4084 non-null   float64
 5   sugar     4084 non-null   float64
 6   bmi       4084 non-null   float64
dtypes: float64(7)
memory usage: 223.5 KB


In [178]:
# Verschillende dataframes benoemen die aangroepen kunnen worden
# To understand the value of genetic, bmi and daily lifechoices
# Default
v1 = ['genetic','exercise','smoking','alcohol','sugar','bmi']

# Default without genetic 'lifestyle'
v2 = ['exercise','smoking','alcohol','sugar','bmi']

# Default without bmi
v3 = ['genetic','exercise','smoking','alcohol','sugar']

# Only lifestyle (not genetic and bmi)
v4 = ['exercise','smoking','alcohol','sugar']

# Only genetic > negatieve intercept
v5 = ['genetic']

# Only BMI

v6 = ['bmi']

version_list = [v1,v2,v3,v4,v5]

In [164]:
def calculateModel(dataframe, version):
    x = dataframe[version]
    y = dataframe.loc[:, 'lifespan']
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.3, random_state = 42)
    
    model = LinearRegression()
    model.fit(x_train,y_train)
    predictions = model.predict(x_test)

    ms_error =  mean_squared_error(y_test, predictions)
    ma_error = mean_absolute_error(y_test, predictions)
    
    coefs = list(zip(x.columns, model.coef_))
    
    r2 = r2_score(y_test, predictions)

    intercept = model.intercept_
    
    return ms_error, ma_error, coefs, r2, intercept

calculateModel(df, v1)

    

(6.693212594542584,
 0.8749815106030503,
 [('genetic', 1.0009704513524607),
  ('exercise', 0.7776985050881321),
  ('smoking', -0.26863618220273244),
  ('alcohol', -0.1975255196844301),
  ('sugar', -0.05445870178572912),
  ('bmi', -0.07155025866396975)],
 0.9036393025342597,
 1.3430756808759128)

In [165]:
gen_coef = 1.0009704513524607
exe_coef = 0.7776985050881321
smo_coef = -0.26863618220273244
alc_coef = -0.1975255196844301
sug_coef = -0.05445870178572912
bmi_coef = -0.07155025866396975

In [177]:
def determineLifespan(genetic, exercise, smoking, alcohol, sugar, bmi):
    
    genetic *= gen_coef
    exercise *= exe_coef
    smoking *= smo_coef
    alcohol *= alc_coef
    sugar *= sug_coef
    bmi *= bmi_coef
    b = 1.3430756808759128
    lifespan = genetic + exercise + smoking + alcohol + sugar + bmi + b
    
    return lifespan

determineLifespan(80, 0, 10, 1, 6.9, 29)

76.08610190378435

In [171]:
df.head()

,genetic,exercise,smoking,alcohol,lifespan,sugar,bmi
0,73.9,0.9,0.0,2.4,73.1,6.9,29.130752
1,86.0,1.8,8.1,0.4,85.0,4.2,35.627366
2,83.3,1.1,0.8,4.6,81.6,7.5,35.963326
3,82.8,4.7,11.8,1.0,81.0,2.9,41.418798
4,78.7,1.5,8.3,4.9,75.0,5.5,22.598157
